In [25]:
from openai import OpenAI
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
my_api_key = "your_openai_api_key_here"

In [24]:
#Prompt Template to Create Realistic Estate Listing
prompt_template = """
You are a professional estate agent. You need to find a random neighborhood/district in the USA and then fill out the template below.

Template:
Neighborhood: {neighborhood}
Price: {price}
Bedrooms: {bedroom_count}
Bathrooms: {bathroom_count}
House Size: {sqft_size}
Description: {estate_description}
Neighborhood Description: {neighborhood_description}

Please ensure each example is unique and realistic.
"""

In [26]:
#Estate List Generating Function, I increased temperature because I need more varied examples
def generate_estate_list(my_key):
    client = OpenAI(api_key = my_key,)
    completion = client.chat.completions.create(
    model = "gpt-4-turbo",
    temperature=0.7,
    max_tokens=2048,
    messages = [
        {"role": "system", "content": prompt_template},
    ]
    )
    return completion.choices[0].message.content.strip()

In [27]:
estate_list = []
# Generating 40 examples
example_estate_count = 40
for _ in range(example_estate_count):
    estate_details = generate_estate_list(my_key=my_api_key)
    estate_list.append(estate_details)

In [53]:
for i,element in enumerate(estate_list[:10]):
    print(i+1,")", element)

1 ) Neighborhood: Laurelhurst
Price: $1,200,000
Bedrooms: 4
Bathrooms: 3
House Size: 3,200 sqft
Description: This stunning two-story home, located in the prestigious Laurelhurst neighborhood, features an open-concept kitchen with stainless steel appliances, granite countertops, and a large island perfect for entertaining. The living room boasts a beautiful fireplace and ample natural light. The master suite includes a spacious walk-in closet and a luxurious en-suite bathroom with a soaking tub and separate shower. Hardwood flooring throughout the house adds elegance and warmth.
Neighborhood Description: Laurelhurst is one of the most sought-after neighborhoods in Seattle, known for its beautiful waterfront parks, top-rated schools, and tight-knit community. The area offers a peaceful residential setting with easy access to downtown Seattle, making it perfect for families and professionals alike. The neighborhood also features a private beach club, tennis courts, and beautifully maintai

In [28]:
# Create Embedding Objects, I choose OpenAI embeddings
embedder = OpenAIEmbeddings(openai_api_key=my_api_key)

In [29]:
#Creating Chroma Vector Database using LLM-base generated estate list and OpenAI Embeddings
db = Chroma.from_texts(estate_list,embedder)

In [30]:
# Parsing Function (Using LLM) to extract and format user input according to specific template
def query_parser(my_key, user_input_text):
    # Define the prompt template with placeholders for dynamic content
    answer_prompt_template = f"""
        Extract and summarize the key information from the user's query about real estate. Focus on identifying and extracting specific details such as the neighborhood,price, number of bedrooms, number of bathrooms, and the size of the house in square feet.

        User Query: "{user_input_text}"

        Example:
        User Query: "I'm looking for a 3-bedroom, 2-bathroom house in the Brooklyn area. It should be under $800,000 and have at least 1500 square feet of space."

        Please provide a summarized version of the query along with the structured details in the following format:

        "Summarized Query: Looking for a house in Brooklyn with 3 bedrooms, 2 bathrooms, priced under $800,000, and at least 1500 sq ft."

        Neighborhood: Brooklyn
        Price: under $800,000
        Bedrooms: 3
        Bathrooms: 2
        House Size: at least 1500 sqft"
    """

    # Initialize the OpenAI client with the provided API key
    client = OpenAI(api_key=my_key)

    # Create a completion request to the OpenAI API with the formatted prompt
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0.7,
        max_tokens=1024,
        messages=[
            {"role": "system", "content": answer_prompt_template},
        ]
    )
    # Return the formatted answer
    return completion.choices[0].message.content.strip()

In [41]:
# Example user input text - plain text
user_input = "I'm looking for a at least 1-bedroom, 2-bathroom house. I am flexible for neighborhood option. It should be under $10000,000 and have at least 500 square feet of space. I need 10 suggestions and all suggestions should be include different neighborhood."
query = query_parser(my_key=my_api_key, user_input_text=user_input)
print(query)

Summarized Query: Looking for a house with at least 1 bedroom, 2 bathrooms, priced under $10,000,000, and at least 500 sq ft, flexible with neighborhood; needs 10 suggestions, each from different neighborhoods.

Neighborhood: Flexible
Price: under $10,000,000
Bedrooms: at least 1
Bathrooms: 2
House Size: at least 500 sqft


In [ ]:
user_input = "Enter your query here"

In [43]:
# TOP 10 Result from Vector Database
similar_docs = db.similarity_search(query, k=10)
#Example result from Vector Database
similar_docs[6].page_content

'Neighborhood: Hyde Park\nPrice: $450,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 2,100 sqft\nDescription: This charming single-family home in the heart of Hyde Park features a spacious open floor plan, hardwood floors, and a modern kitchen equipped with stainless steel appliances and granite countertops. The master suite offers a large walk-in closet and en-suite bathroom. A beautifully landscaped backyard includes a patio area perfect for outdoor dining and entertaining.\nNeighborhood Description: Hyde Park is known for its rich history and vibrant community atmosphere. It boasts an array of local shops, cafes, and parks. The neighborhood is also home to several top-rated schools and has easy access to downtown via public transportation, making it an ideal location for families and professionals alike.'

In [44]:
# Funtion for formatting final answer in a generative way by including estate information that created by another LLM in the previous cells. We create final answer to present buyer in a natural estate agent language
def generate_estate_answers(my_key, user_query, top1_answer, top2_answer, top3_answer, top4_answer, top5_answer,top6_answer,top7_answer,top8_answer,top9_answer,top10_answer):
    # Define the prompt template with placeholders for dynamic content
    answer_prompt_template = f"""
    You are a professional estate agent. The list you given you should explain every item in the list with a fancy way to sell estate to buyer. Augment and beautify your answer with {user_query} and
     1) {top1_answer}
     2) {top2_answer}
     3) {top3_answer}
     4) {top4_answer}
     5) {top5_answer}
     6) {top6_answer}
     7) {top7_answer}
     8) {top8_answer}
     9) {top9_answer}
     10) {top10_answer}

    Please ensure each answer is realistic.
    """

    # Initialize the OpenAI client with the provided API key
    client = OpenAI(api_key=my_key)

    # Create a completion request to the OpenAI API with the formatted prompt
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0.7,
        max_tokens=2048,
        messages=[
            {"role": "system", "content": answer_prompt_template},
        ]
    )

    # Return the formatted answer
    return completion.choices[0].message.content.strip()

In [45]:
#getting top-10 content and store in the array
top_10_estate = []
for i in range(len(similar_docs)):
    top_10_estate.append(similar_docs[i].page_content)

In [46]:
# Generating final answer
result_text = generate_estate_answers(my_api_key,query,top_10_estate[0],top_10_estate[1],top_10_estate[2],top_10_estate[3],top_10_estate[4],top_10_estate[5],top_10_estate[6],top_10_estate[7],top_10_estate[8],top_10_estate[9])

In [47]:
# Display final answer
print(result_text)

Certainly! Here are ten beautifully curated homes from different neighborhoods tailored to your preferences:

1) **Park Slope, Brooklyn, NY**
   - **Price:** $1,200,000
   - **Bedrooms:** 3
   - **Bathrooms:** 2
   - **House Size:** 1,800 sqft
   - **Description:** Step into this charming brownstone townhouse, boasting original architectural details, a modern kitchen, and an enchanting backyard garden.
   - **Neighborhood Description:** Park Slope offers a warm, family-friendly environment with tree-lined streets, trendy boutiques, and close proximity to the lush expanses of Prospect Park.

2) **Pacific Heights, San Francisco, CA**
   - **Price:** $3,500,000
   - **Bedrooms:** 4
   - **Bathrooms:** 3.5
   - **House Size:** 2,500 sqft
   - **Description:** Experience elegance in this Victorian home featuring panoramic views of the Golden Gate Bridge, a gourmet kitchen, and expansive living areas perfect for entertaining.
   - **Neighborhood Description:** Pacific Heights, known for its 